# Behavior Cloningを使用して実機の自動運転の学習をしたい。
学習データは、自分が運転したデータを使用する。
データの形式は、画像とその時の操作であるsteeringとthrottleの値が保存されている。


In [1]:
import numpy as np
import gymnasium
import gym
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper

from gym.spaces import Box
from gym.envs.registration import register
from MyEnv.config import config as conf
from MyEnv.env import MyEnv
import os
import json
from PIL import Image


/home/serpens22/miniconda3/envs/env38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 環境を事務のレジストリに追加する

In [2]:
gym.register(
    id = 'MyEnv-v0',
    entry_point = 'MyEnv.env:MyEnv',
    max_episode_steps = 1000,
)


ベクトル化された環境を作成する

In [3]:
# expert_data_path = '/path/to/data/tub_9_24-01-09'
# env = MyEnv(expert_data_path)

In [28]:
import os
import json
import numpy as np
from PIL import Image

def load_expert_data(data_path):
    # expert_dataの初期化
    expert_data = {'images': [], 'actions': []}
    print(expert_data)
    count = 0

    # 画像ファイルに対応するJSONファイルを取得
    json_file_list = [json_file for json_file in os.listdir(data_path) if json_file.startswith('record_') and json_file.endswith('.json')]
    print(len(json_file_list))

    for json_file in json_file_list:
        count += 1
        print(count)
        print(json_file)

        # レコードのファイルパスを構築
        json_path = os.path.join(data_path, json_file)
        print(json_path)

        # レコードの読み込み
        try:
            with open(json_path, 'r') as json_file:
                record_data = json.load(json_file)
        except FileNotFoundError:
            print(f"エラー：{json_path} でJSONファイルが見つかりませんでした。")
            continue
        except json.JSONDecodeError:
            print(f"エラー：{json_path} のJSONファイルのデコードに失敗しました。")
            continue

        # 画像データの読み込み
        image_file = record_data.get('image_file', '')  # 画像ファイル名をJSONから取得
        image_path = os.path.join(data_path, image_file)
        try:
            image_data = np.array(Image.open(image_path))
        except FileNotFoundError:
            print(f"エラー：{image_path} で画像ファイルが見つかりませんでした。")
            continue

        # expert_dataに追加
        expert_data['images'].append(image_data)
        expert_data['actions'].append([record_data.get('user/angle', 0), record_data.get('user/throttle', 0)])

    return expert_data


In [29]:
expert_data_path = './data/tub_9_24-01-09'
expert_data = load_expert_data(expert_data_path)
print(expert_data['actions'])
env = gymnasium.make('MyEnv-v0', expert_data)

{'images': [], 'actions': []}
3479
1
record_1589.json
./data/tub_9_24-01-09/record_1589.json


IsADirectoryError: [Errno 21] Is a directory: './data/tub_9_24-01-09/'

In [7]:
venv = make_vec_env(
    "MyEnv-v0",
    rng = np.random.default_rng(),
    n_envs = 4,
    post_wrappers = [lambda env, _: RolloutInfoWrapper(env)],
)

NameNotFound: Environment `MyEnv` doesn't exist.